<a href="https://colab.research.google.com/github/AslanDevbrat/AwirosProject/blob/master/notebooks/mmaction2_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/AWIROS/mmlab/mmaction2/


/content/drive/MyDrive/AWIROS/mmlab/mmaction2


In [3]:
!ls

checkpoints  docs	  mmaction	      README_zh-CN.md	setup.cfg
configs      docs_zh_CN   mmaction2.egg-info  requirements	setup.py
demo	     LICENSE	  model-index.yml     requirements.txt	tests
docker	     MANIFEST.in  README.md	      resources		tools


# MMAction2 Tutorial

Welcome to MMAction2! This is the official colab tutorial for using MMAction2. In this tutorial, you will learn
- Perform inference with a MMAction2 recognizer.
- Train a new recognizer with a new dataset.

Let's start!

## Install MMAction2

In [4]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.8.0+cu101 torchvision==0.9.0+cu101 torchtext==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==1.3.9 -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.0/index.html

# Install mmaction2
#!rm -rf mmaction2
#!git clone https://github.com/open-mmlab/mmaction2.git
#%cd mmaction2

!pip install -e .

# Install some optional requirements
!pip install -r requirements/optional.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 7.9 kB/s 
     |████████████████████████████████| 17.3 MB 156 kB/s 
     |████████████████████████████████| 7.1 MB 4.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0
Looking in links: https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.0/index.html
     |████████████████████████████████| 31.4 MB 26 kB/s 
     |████████████████████████████████| 185 kB 5.0 MB/s 
Obtaining file:///content/driv

In [4]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.8.0+cu101 False
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
0.16.0
10.1
GCC 7.3


## Perform inference with a MMAction2 recognizer
MMAction2 already provides high level APIs to do inference and training.

In [20]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
      -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth

--2021-07-30 12:48:03--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.78
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97579339 (93M) [application/octet-stream]
Saving to: ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’

checkpoints/tsn_r50 100%[===================>]  93.06M  12.5MB/s    in 8.3s    

2021-07-30 12:48:12 (11.2 MB/s) - ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’ saved [97579339/97579339]



In [52]:
!VERSION=2.2 bash download_annotations.sh

/content/drive/MyDrive/AWIROS/mmlab/mmaction2/tools/data/ava/annotations does not exist. Creating
--2021-07-30 13:18:31--  https://research.google.com/ava/download/ava_v2.2.zip
Resolving research.google.com (research.google.com)... 142.250.101.138, 142.250.101.102, 142.250.101.100, ...
Connecting to research.google.com (research.google.com)|142.250.101.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7721406 (7.4M) [application/zip]
Saving to: ‘ava_v2.2.zip’

ava_v2.2.zip        100%[===================>]   7.36M  17.6MB/s    in 0.4s    

2021-07-30 13:18:31 (17.6 MB/s) - ‘ava_v2.2.zip’ saved [7721406/7721406]

Archive:  ava_v2.2.zip
  inflating: /content/drive/MyDrive/AWIROS/mmlab/mmaction2/tools/data/ava/annotations/ava_action_list_v2.2_for_activitynet_2019.pbtxt  
  inflating: /content/drive/MyDrive/AWIROS/mmlab/mmaction2/tools/data/ava/annotations/ava_action_list_v2.2.pbtxt  
  inflating: /content/drive/MyDrive/AWIROS/mmlab/mmaction2/tools/data/ava/anno

In [39]:
%cd /content/drive/MyDrive/AWIROS/mmlab/mmaction2/tools/data/ava/annotations/

/content/drive/MyDrive/AWIROS/mmlab/mmaction2/tools/data/ava/annotations


In [50]:
!pwd

/content/drive/My Drive/AWIROS/mmlab/mmaction2/tools/data/ava


In [54]:
%cd ava/
!pwd

[Errno 2] No such file or directory: 'ava/'
/content/drive/My Drive/AWIROS/mmlab/mmaction2/tools/data/ava
/content/drive/MyDrive/AWIROS/mmlab/mmaction2/tools/data/ava


In [58]:
!rm -R videos/

rm: cannot remove 'videos/': No such file or directory


In [ ]:

!bash /content/drive/MyDrive/AWIROS/mmlab/mmaction2/tools/data/ava/download_videos.sh --tmpdir /content/drive/MyDrive/AWIROS/Datasets

In [62]:
!pwd
%cd videos/

/content/drive/My Drive/AWIROS/mmlab/mmaction2/tools/data/ava
/content/drive/My Drive/AWIROS/mmlab/mmaction2/tools/data/ava/videos


In [63]:
ls | wc -l

299


In [65]:
%cd /content/drive/MyDrive/AWIROS/mmlab/mmaction2

/content/drive/MyDrive/AWIROS/mmlab/mmaction2


In [3]:
from mmaction.apis import inference_recognizer, init_recognizer

# Choose to use a config and initialize the recognizer
config = 'configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics700_rgb.py'
# Setup a checkpoint file to load
checkpoint = 'https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics700_rgb/tsn_r50_video_1x1x8_100e_kinetics700_rgb_20201015-e381a6c7.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0')

Use load_from_http loader


In [3]:
pwd

'/content/drive/MyDrive/AWIROS/mmlab/mmaction2'

In [7]:
!python demo/long_video_demo.py configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics700_rgb.py \
  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics700_rgb/tsn_r50_video_1x1x8_100e_kinetics700_rgb_20201015-e381a6c7.pth demo/biocon-2.avi demo/label_map_k700.txt demo/biocon_output_k700.mp4 --input-step 30

Use load_from_http loader
[  ] 553/2847, 1.4 task/s, elapsed: 390s, ETA:  1618s[msvideo1 @ 0x56060f21f180]  MS Video-1 warning: stream_ptr out of bounds (17644 >= 17638)
[>>] 2847/2847, 7.3 task/s, elapsed: 392s, ETA:     0s

In [77]:
mv /content/mmaction2/demo/biocon-2.avi /content/drive/MyDrive/AWIROS/mmlab/mmaction2/demo

In [ ]:
# Use the recognizer to do inference
video = 'demo/biocon-2.avi'
label = 'demo/label_map_k700.txt'
results = inference_recognizer(model, video, label)

In [75]:
# Let's show the results
for result in results:
    print(f'{result[0]}: ', result[1])

writing.:  40.184624
giving something to other person.:  11.104316
stretch oneself.:  7.912055
nod head/bow.:  7.853262
throw.:  7.015127


## Train a recognizer on customized dataset

To train a new recognizer, there are usually three things to do:
1. Support a new dataset
2. Modify the config
3. Train a new recognizer

### Support a new dataset

In this tutorial, we gives an example to convert the data into the format of existing datasets. Other methods and more advanced usages can be found in the [doc](/docs/tutorials/new_dataset.md)

Firstly, let's download a tiny dataset obtained from [Kinetics-400](https://deepmind.com/research/open-source/open-source-datasets/kinetics/). We select 30 videos with their labels as train dataset and 10 videos with their labels as test dataset.

In [ ]:
# download, decompress the data
!rm kinetics400_tiny.zip*
!rm -rf kinetics400_tiny
!wget https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
!unzip kinetics400_tiny.zip > /dev/null

rm: cannot remove 'kinetics400_tiny.zip*': No such file or directory
--2021-07-11 12:44:29--  https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.78
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18308682 (17M) [application/zip]
Saving to: ‘kinetics400_tiny.zip’

kinetics400_tiny.zi 100%[===================>]  17.46M  10.7MB/s    in 1.6s    

2021-07-11 12:44:31 (10.7 MB/s) - ‘kinetics400_tiny.zip’ saved [18308682/18308682]



In [ ]:
# Check the directory structure of the tiny data

# Install tree first
!apt-get -q install tree
!tree kinetics400_tiny

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (88.7 kB/s)
Selecting previously unselected package tree.
(Reading database ... 160815 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
kinetics400_tiny
├── kinetics_tiny_train_video.txt
├── kinetics_tiny_val_video.txt
├── train
│   ├── 27_CSXByd3s.mp4
│   ├── 34XczvTaRiI.mp4
│   ├── A-wiliK50Zw.mp4
│   ├── D32_1gwq35E.mp4
│   ├── D92m0HsHjcQ.mp4
│   ├── DbX8mPslRXg.mp4
│   ├── FMlSTTpN3VY.mp4
│   ├── h10B9SVE-nk.mp4
│   ├── 

In [ ]:
# After downloading the data, we need to check the annotation format
!cat kinetics400_tiny/kinetics_tiny_train_video.txt

D32_1gwq35E.mp4 0
iRuyZSKhHRg.mp4 1
oXy-e_P_cAI.mp4 0
34XczvTaRiI.mp4 1
h2YqqUhnR34.mp4 0
O46YA8tI530.mp4 0
kFC3KY2bOP8.mp4 1
WWP5HZJsg-o.mp4 1
phDqGd0NKoo.mp4 1
yLC9CtWU5ws.mp4 0
27_CSXByd3s.mp4 1
IyfILH9lBRo.mp4 1
T_TMNGzVrDk.mp4 1
TkkZPZHbAKA.mp4 0
PnOe3GZRVX8.mp4 1
soEcZZsBmDs.mp4 1
FMlSTTpN3VY.mp4 1
WaS0qwP46Us.mp4 0
A-wiliK50Zw.mp4 1
oMrZaozOvdQ.mp4 1
ZQV4U2KQ370.mp4 0
DbX8mPslRXg.mp4 1
h10B9SVE-nk.mp4 1
P5M-hAts7MQ.mp4 0
R8HXQkdgKWA.mp4 0
D92m0HsHjcQ.mp4 0
RqnKtCEoEcA.mp4 0
LvcFDgCAXQs.mp4 0
xGY2dP0YUjA.mp4 0
Wh_YPQdH1Zg.mp4 0


According to the format defined in [`VideoDataset`](./datasets/video_dataset.py), each line indicates a sample video with the filepath and label, which are split with a whitespace.

### Modify the config

In the next step, we need to modify the config for the training.
To accelerate the process, we finetune a recognizer using a pre-trained recognizer.

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

Given a config that trains a TSN model on kinetics400-full dataset, we need to modify some values to use it for training TSN on Kinetics400-tiny dataset.


In [ ]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'kinetics400_tiny/train/'
cfg.data_root_val = 'kinetics400_tiny/val/'
cfg.ann_file_train = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.ann_file_val = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.ann_file_test = 'kinetics400_tiny/kinetics_tiny_val_video.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.test.data_prefix = 'kinetics400_tiny/val/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.data.train.data_prefix = 'kinetics400_tiny/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.val.data_prefix = 'kinetics400_tiny/val/'

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 10

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 5
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# Save the best
cfg.evaluation.save_best='auto'


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='Recognizer2D',
    backbone=dict(
        type='ResNet',
        pretrained='torchvision://resnet50',
        depth=50,
        norm_eval=False),
    cls_head=dict(
        type='TSNHead',
        num_classes=2,
        in_channels=2048,
        spatial_type='avg',
        consensus=dict(type='AvgConsensus', dim=1),
        dropout_ratio=0.4,
        init_std=0.01),
    train_cfg=None,
    test_cfg=dict(average_clips=None))
optimizer = dict(type='SGD', lr=7.8125e-05, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=40, norm_type=2))
lr_config = dict(policy='step', step=[40, 80])
total_epochs = 10
checkpoint_config = dict(interval=5)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
resume_from = None
workflow = [('train', 1)]
dataset_type = 'VideoDataset'
data_root = 

### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [ ]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

Use load_from_torchvision loader


2021-07-11 13:00:46,931 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.bias', 'fc.weight'}
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
2021-07-11 13:00:46,980 - mmaction - INFO - load checkpoint from ./checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
2021-07-11 13:00:46,981 - mmaction - INFO - Use load_from_local loader
2021-07-11 13:00:47,071 - mmaction - WARNING - The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2048]) from checkpoint,

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 4.9 task/s, elapsed: 2s, ETA:     0s

2021-07-11 13:01:21,673 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-07-11 13:01:21,677 - mmaction - INFO - 
top1_acc	0.7000
top5_acc	1.0000
2021-07-11 13:01:21,679 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-07-11 13:01:21,682 - mmaction - INFO - 
mean_acc	0.7000
2021-07-11 13:01:22,264 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_5.pth.
2021-07-11 13:01:22,267 - mmaction - INFO - Best top1_acc is 0.7000 at 5 epoch.
2021-07-11 13:01:22,271 - mmaction - INFO - Epoch(val) [5][5]	top1_acc: 0.7000, top5_acc: 1.0000, mean_class_accuracy: 0.7000
2021-07-11 13:01:26,623 - mmaction - INFO - Epoch [6][5/15]	lr: 7.813e-05, eta: 0:00:31, time: 0.868, data_time: 0.656, memory: 2918, top1_acc: 0.7000, top5_acc: 1.0000, loss_cls: 0.6753, loss: 0.6753, grad_norm: 11.8640
2021-07-11 13:01:27,597 - mmaction - INFO - Epoch [6][10/15]	lr: 7.813e-05, eta: 0:00:28, time: 0.195, data_time: 0.003, memory: 2918, top1_acc: 0.6000, top5_acc: 1.0000, loss_cls

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 5.9 task/s, elapsed: 2s, ETA:     0s

2021-07-11 13:01:56,741 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-07-11 13:01:56,743 - mmaction - INFO - 
top1_acc	1.0000
top5_acc	1.0000
2021-07-11 13:01:56,749 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-07-11 13:01:56,750 - mmaction - INFO - 
mean_acc	1.0000
2021-07-11 13:01:57,267 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_10.pth.
2021-07-11 13:01:57,269 - mmaction - INFO - Best top1_acc is 1.0000 at 10 epoch.
2021-07-11 13:01:57,270 - mmaction - INFO - Epoch(val) [10][5]	top1_acc: 1.0000, top5_acc: 1.0000, mean_class_accuracy: 1.0000


### Understand the log
From the log, we can have a basic understanding the training process and know how well the recognizer is trained.

Firstly, the ResNet-50 backbone pre-trained on ImageNet is loaded, this is a common practice since training from scratch is more cost. The log shows that all the weights of the ResNet-50 backbone are loaded except the `fc.bias` and `fc.weight`.

Second, since the dataset we are using is small, we loaded a TSN model and finetune it for action recognition.
The original TSN is trained on original Kinetics-400 dataset which contains 400 classes but Kinetics-400 Tiny dataset only have 2 classes. Therefore, the last FC layer of the pre-trained TSN for classification has different weight shape and is not used.

Third, after training, the recognizer is evaluated by the default evaluation. The results show that the recognizer achieves 100% top1 accuracy and 100% top5 accuracy on the val dataset,
 
Not bad!

## Test the trained recognizer

After finetuning the recognizer, let's check the prediction results!

In [ ]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel

# Build a test dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

eval_config = cfg.evaluation
eval_config.pop('interval')
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[                                                  ] 0/10, elapsed: 0s, ETA:

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 2.2 task/s, elapsed: 5s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	1.0000
top5_acc	1.0000

Evaluating mean_class_accuracy ...

mean_acc	1.0000
top1_acc: 1.0000
top5_acc: 1.0000
mean_class_accuracy: 1.0000


/content/mmaction2/mmaction/datasets/base.py:166: UserWarning: Option arguments for metrics has been changed to `metric_options`, See 'https://github.com/open-mmlab/mmaction2/pull/286' for more details
  'Option arguments for metrics has been changed to '
